In [ ]:
device = 'cuda'

import torch
import bigvgan
import librosa as li
from meldataset import get_mel_spectrogram
import logging

: 

In [ ]:
!pip install "numpy<2"

In [ ]:

# instantiate the model. You can optionally set use_cuda_kernel=True for faster inference.
model = bigvgan.BigVGAN.from_pretrained('nvidia/bigvgan_v2_24khz_100band_256x', use_cuda_kernel=False)

# remove weight norm in the model and set to eval mode
model.remove_weight_norm()
model = model.eval().to(device)

# # load wav file and compute mel spectrogram
# wav_path = '/home/mila/k/krishna-maneesha.dendukuri/scratch/gamadhani/datasets/merged_data/audio/Bajeri_Mori_Payal_Jhanana_and_Lat_Ulajhi_Sulajha_Ja_Balam_82d9adb4-ad60-4437-b3ed-a5dda2a77f61/vocals.mp3'
# wav, sr = librosa.load(wav_path, sr=model.h.sampling_rate, mono=True, offset=600, duration=10) # wav is np.ndarray with shape [T_time] and values in [-1, 1]
# wav = torch.FloatTensor(wav).unsqueeze(0) # wav is FloatTensor with shape [B(1), T_time]

In [ ]:
model.h.sampling_rate

In [ ]:
wav_path = '/home/mila/k/krishna-maneesha.dendukuri/scratch/gamadhani/datasets/final_dataset/audio/Bajeri_Mori_Payal_Jhanana_and_Lat_Ulajhi_Sulajha_Ja_Balam_82d9adb4-ad60-4437-b3ed-a5dda2a77f61/vocals.mp3'
wav, sr = li.load(wav_path, sr=model.h.sampling_rate, mono=True, offset=600, duration=12) # wav is np.ndarray with shape [T_time] and values in [-1, 1]
# wav, sr = li.load(wav_path, sr=16000, mono=True, offset=600, duration=12) # wav is np.ndarray with shape [T_time] and values in [-1, 1]


In [ ]:
from IPython.display import Audio
Audio(wav, rate=sr)

In [43]:
from scipy.io.wavfile import write

write("/home/mila/k/krishna-maneesha.dendukuri/BigVGAN/test_inpy.wav", sr, wav)

In [44]:
wav = torch.Tensor(wav)
wav.shape

torch.Size([288000])

## 1. GaMaDHaNi Mel-to-STFT Magnitude Spectrogram Conversion

In [26]:
NFFT = 1024
NUM_MELS = 192
SR = 16000

In [27]:
def torch_stft(x, nfft):
    window = torch.hann_window(nfft).to(x)
    x = torch.stft(
        x,
        n_fft=nfft,
        hop_length=nfft // 4,
        win_length=nfft,
        window=window,
        center=True,
        return_complex=True,
    )
    x = 2 * x / torch.mean(window)
    return x
    
def to_mels(stft, nfft, num_mels, sr, eps=1e-2, fmin=40, alt_logic = False, C=1):
    mels = li.filters.mel(
        sr=sr,
        n_fft=nfft,
        n_mels=num_mels,
        fmin=fmin,
    )
    mels = torch.from_numpy(mels).to(stft)
    mel_stft = torch.einsum("mf,bft->bmt", mels, stft)
    if alt_logic:
        mel_stft = torch.log(torch.clamp(mel_stft, min=eps) * C)
    else:
        mel_stft = torch.log(mel_stft + eps)
    return mel_stft

def normalize(x, qt=None):
    x_flat = x.reshape(-1, 1)
    if qt is None:
        logging.warning('No quantile transformer found, returning input')
        return x
    return torch.Tensor(qt.transform(x_flat).reshape(x.shape))

def audio_to_normalized_mels(x, nfft, num_mels, sr, qt):
    stfts = torch_stft(x, nfft=nfft).abs()[..., :-1]
    print(stfts.shape)
    mel_stfts = to_mels(stfts, nfft, num_mels, sr)
    print(mel_stfts.shape)
    return normalize(mel_stfts, qt).to(x)

def unnormalize(x, qt=None):
    x_flat = x.reshape(-1, 1)
    if qt is None:
        logging.warning('No quantile transformer found, returning input')
        return x
    if isinstance(x_flat, torch.Tensor):
        x_flat = x_flat.detach().cpu().numpy()
    return torch.Tensor(qt.inverse_transform(x_flat).reshape(x.shape))

def from_mels(mel_stft, nfft, num_mels, sr, eps=1e-2):
    mels = li.filters.mel(
        sr=sr,
        n_fft=nfft,
        n_mels=num_mels,
        fmin=40,
    )
    mels = torch.from_numpy(mels).to(mel_stft)
    mels = torch.pinverse(mels)
    mel_stft = torch.exp(mel_stft) - eps
    stft = torch.einsum("fm,bmt->bft", mels, mel_stft)
    return stft



In [28]:
# add btch dimension to wav
batched_wav = wav.unsqueeze(0)

In [29]:
mels = audio_to_normalized_mels(batched_wav, nfft=NFFT, num_mels=NUM_MELS, sr=SR, qt=None)

mels = unnormalize(mels, qt=None)
stft = from_mels(mels, nfft=NFFT, num_mels=NUM_MELS, sr=SR)


torch.Size([1, 513, 750])
torch.Size([1, 192, 750])


In [30]:
mels.shape, stft.shape

(torch.Size([1, 192, 750]), torch.Size([1, 513, 750]))

## 2. STFT Magnitude Spectrogram-to-BigVGAN Mel Spectrogram Conversion

In [31]:
NFFT = model.h.n_fft
NUM_MELS = model.h.num_mels
SR = model.h.sampling_rate

In [32]:
NFFT, model.h.win_size, model.h.hop_size, model.h.fmin, model.h.fmax

(1024, 1024, 256, 0, None)

In [33]:
bigvgan_mels = to_mels(stft, nfft=NFFT, num_mels=NUM_MELS, sr=SR, eps=1e-5, fmin=0, alt_logic=True)
bigvgan_mels.shape

torch.Size([1, 100, 750])

In [34]:
bigvgan_mels = bigvgan_mels.to(device)

## 3. BigVGAN Inference

In [46]:
wav.shape

torch.Size([288000])

In [47]:
# # compute mel spectrogram from the ground truth audio
mel = get_mel_spectrogram(wav.unsqueeze(0), model.h).to(device) # mel is FloatTensor with shape [B(1), C_mel, T_frame]

# generate waveform from mel
with torch.inference_mode():
    wav_gen = model(bigvgan_mels) # wav_gen is FloatTensor with shape [B(1), 1, T_time] and values in [-1, 1]
wav_gen_float = wav_gen.squeeze(0).cpu() # wav_gen is FloatTensor with shape [1, T_time]

# you can convert the generated waveform to 16 bit linear PCM
wav_gen_int16 = (wav_gen_float * 32767.0).numpy().astype('int16') # wav_gen is now np.ndarray with shape [1, T_time] and int16 dtype

In [48]:
mel.shape

torch.Size([1, 100, 1125])

In [49]:
wav_gen_float.shape

torch.Size([1, 192000])

In [50]:
Audio(wav_gen_float, rate=sr)

## Testing with generated sample

In [23]:
gen_stft = torch.load("/home/mila/k/krishna-maneesha.dendukuri/BigVGAN/bigvgan_tests/mels.pt", map_location=device)

In [24]:
gen_stft.shape

torch.Size([2, 513, 750])

In [25]:
gen_bigvgan_mels = to_mels(gen_stft, nfft=NFFT, num_mels=NUM_MELS, sr=SR, eps=1e-5, fmin=0, alt_logic=True).to(device)


In [26]:
# # compute mel spectrogram from the ground truth audio
# mel = get_mel_spectrogram(wav, model.h).to(device) # mel is FloatTensor with shape [B(1), C_mel, T_frame]

# generate waveform from mel
with torch.inference_mode():
    wav_gen = model(gen_bigvgan_mels) # wav_gen is FloatTensor with shape [B(1), 1, T_time] and values in [-1, 1]
wav_gen_float = wav_gen.squeeze(0).cpu() # wav_gen is FloatTensor with shape [1, T_time]

# you can convert the generated waveform to 16 bit linear PCM
wav_gen_int16 = (wav_gen_float * 32767.0).numpy().astype('int16') # wav_gen is now np.ndarray with shape [1, T_time] and int16 dtype

In [27]:
Audio(wav_gen_float[0], rate=sr)

In [28]:
Audio(wav_gen_float[1], rate=sr)

In [61]:
wav_gen_float.shape, gen_bigvgan_mels.shape

(torch.Size([2, 1, 192000]), torch.Size([2, 100, 750]))